In [1]:
import pandas as pd

df = pd.read_parquet('../raw_data/filtered_df.parquet')

In [2]:
df = df[df['nutriscore_grade'].isin(['a', 'b', 'c', 'd', 'e'])]

In [3]:
df['y'] = df["nutriscore_score"] + (df["nova_group"] * 10)

In [4]:
df = df.drop(columns=['labels_en'])
df['text'] = (df['ingredients_text'] + ' ' + df['additives_en']).str.strip()
df['text'] = df['text'].str.replace(r',([^ ])', r', \1', regex=True) # only replace those commas without following spaces

In [5]:
df = df.drop(columns=['ingredients_text', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group'])
df['other_carbohydrates_100g'] = df['carbohydrates_100g'] - df['fiber_100g'] - df['sugars_100g']
df['other_fat_100g'] = df['fat_100g'] - df['saturated-fat_100g'] - df['trans-fat_100g']
df = df.drop(columns=['fat_100g', 'carbohydrates_100g'])

In [6]:
import os

save_dir = "../shared_data"
os.makedirs(save_dir, exist_ok=True)

df.to_parquet(os.path.join(save_dir, "df.parquet"))

# If you want to save it here:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='y'), df.y, test_size=0.2, random_state=42)

X_train.to_parquet(os.path.join(save_dir, "X_train.parquet"))
X_test.to_parquet(os.path.join(save_dir, "X_test.parquet"))
y_train.to_frame(name="y_train").to_parquet(os.path.join(save_dir, "y_train.parquet"))
y_test.to_frame(name="y_test").to_parquet(os.path.join(save_dir, "y_test.parquet"))